In [20]:
import h5py
import math
import random
import numpy as np
import seaborn as sns
import scipy.signal as signal
import scipy.stats as stats
import matplotlib.pyplot as plt
from scipy.stats import invgamma

In [21]:
SHM_Data_Locate = "D://SHM-Data/"

locate1 = "20180605T150315_D9/pc_f100kHz.h5"
file1 = h5py.File(SHM_Data_Locate + locate1, 'r')
catch1 = file1['/pitchcatch/catch'][()]

locate2 = "20180604T164628_baseline_1/pc_f100kHz.h5"
file2 = h5py.File(SHM_Data_Locate + locate2, 'r')
catch2 = file2['/pitchcatch/catch'][()]

In [22]:
# 板子的大小
plant_size = [500, 500]

# 12组传感器的xy坐标
sensor_locate = np.array([[450, 470], [370, 470], [290, 470], [210, 470], [130, 470], [50, 470], [450, 30], [370, 30], [290, 30], [210, 30], [130, 30], [50, 30]])

# 28个缺陷的位置
damage_locate = np.array([
    [50, 415], [65, 415], [50, 400], [65, 400],
    [250, 427], [265, 427], [250,412], [265, 412],
    [180, 345], [195, 345], [180,330], [195, 345],
    [320, 275], [335, 275], [320, 260], [335, 275],
    [130, 195], [145, 195], [130, 180], [130, 180],
    [435, 205], [450, 205], [435, 190], [450, 190],
    [250, 90], [265, 90], [250, 75], [265, 75]
])

# 因子B
b = 1.05

In [23]:
def aij(aij_catch1, aij_catch2):
    mu1 = aij_catch1.mean()
    mu2 = aij_catch2.mean()
    k = aij_catch1.size
    cxy = 0
    for i in range(k):
        cxy += (aij_catch1[i] - mu1) * (aij_catch2[i] - mu2)
    sigmax = 0
    sigmay = 0
    for i in range(k):
        sigmax += np.square(aij_catch1[i] - mu1)
        sigmay += np.square(aij_catch2[i] - mu2)
    sigmaxy = np.sqrt(sigmax) * np.sqrt(sigmay)
    rho = cxy / sigmaxy
    return 1 - rho

In [ ]:
def RDij(i, j, locate):
    locate_i = sensor_locate[i]
    locate_j = sensor_locate[j]
    RD_ij = (np.sqrt(np.square(locate[0] - locate_i[0]) + np.square(locate[1] - locate_i[1])) + np.sqrt(np.square(locate[0] - locate_j[0]) + np.square(locate[1] - locate_j[1]))) / np.sqrt(np.square(locate_i[0] - locate_j[0]) + np.square(locate_i[1] - locate_j[1]))
    return RD_ij

In [ ]:
def Rij(i, j, locate):
    RD_ij = RDij(i, j, locate)
    if RD_ij < b:
        return RD_ij
    else:
        return b

In [24]:
catch1_1_12 = catch1[10, :]
catch2_1_12 = catch2[10, :]
aij_12 = aij(catch1_1_12, catch2_1_12)